In [1]:
from fastai.vision.all import *
from fastai.metrics import accuracy, Precision, Recall, F1Score
import pandas as pd

In [2]:
DATA_PATH = '/kaggle/input/aptos2019-blindness-detection'

In [3]:
train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))

In [4]:
train['file_path'] = train['id_code'].apply(lambda x: os.path.join(DATA_PATH, 'train_images', f'{x}.png'))

In [5]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=ColReader('file_path'),
    get_y=ColReader('diagnosis'),
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=Resize(256),
    batch_tfms=aug_transforms(size=224) 
)

In [7]:
dls = dblock.dataloaders(train, bs=16)
learn = vision_learner(dls, resnet50,
                       loss_func=CrossEntropyLossFlat(),
                       metrics=[accuracy, F1Score(average='macro')],
                       wd=1e-2) 
learn.fine_tune(10)

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.316892,0.949113,0.709016,0.491177,03:52


epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.965571,0.694523,0.758197,0.495156,03:53
1,0.834615,0.590915,0.775956,0.532045,03:05
2,0.739009,0.527032,0.812842,0.581884,03:05
3,0.687485,0.527014,0.788251,0.586426,03:07
4,0.596271,0.473870,0.825137,0.603671,03:05
5,0.547474,0.464563,0.836066,0.671540,03:05
6,0.515443,0.472582,0.830601,0.642685,03:05
7,0.434934,0.467298,0.836066,0.668580,03:05
8,0.390319,0.457799,0.838798,0.662904,03:04
9,0.349106,0.445691,0.844262,0.668007,03:07


In [8]:
learn.save('resnet50_model')

Path('models/resnet50_model.pth')